# AC - CV vs Overfit

In [1]:
import numpy as np
random_state = 42
np.random.seed(random_state)
import os, glob
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from quantification.metrics import binary_kl_divergence, absolute_error
from quantification.utils.validation import create_bags_with_multiple_prevalence

import seaborn as sns
import matplotlib.pyplot as plt

## Dataset

In [2]:
datasets_dir = "datasets"
dataset_files = [file for file in glob.glob(os.path.join(datasets_dir, "*.csv")) if "k9" not in file]
dataset_names = [os.path.split(name)[-1][:-4] for name in dataset_files]
print("There are a total of {} datasets.".format(len(dataset_names)))

There are a total of 43 datasets.


In [3]:
n_datasets = len(dataset_names)

columns=['dataset', 'method', 'truth', 'predictions', 'kld', 'mae', 'tpr', 'fpr']
errors_df = pd.DataFrame(columns=columns)

## Utils

#### Standard scale data

In [4]:
from sklearn.preprocessing import StandardScaler

def normalize(X_train, X_test):
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test


#### Load data

In [5]:
def load_data(dfile):
    df = pd.read_csv(dfile, header=None)
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values.astype(np.int)
    if -1 in np.unique(y):
        y[y == -1] = 0
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=random_state)
    
    X_train, X_test = normalize(X_train, X_test)
    
    return X_train, X_test, y_train, y_test 

## AC - 50 folds

In [6]:
from quantification.cc import AC

ac = AC(estimator_class=RandomForestClassifier(n_estimators=50, max_depth=5))
for dname, dfile in tqdm(zip(dataset_names, dataset_files), total=n_datasets):
    
    X_train, X_test, y_train, y_test = load_data(dfile)
    
    ac.fit(X_train, y_train, cv=50)
    for X_test_, y_test_, prev_true, in create_bags_with_multiple_prevalence(X_test, y_test, 100):
        prev_true = prev_true[1]
        prev_pred = ac.predict(X_test_)[1]

        kld = binary_kl_divergence(prev_true, prev_pred)
        mae = absolute_error(prev_true, prev_pred)
 
        errors_df = errors_df.append(pd.DataFrame([[dname, 'AC50folds', prev_true, prev_pred, kld, mae, ac.tpr_[1], ac.fpr_[1]]], columns=columns))

 67%|██████▋   | 29/43 [03:08<01:31,  6.50s/it]/usr/local/lib/python3.5/dist-packages/quantification/cc/base.py:307: RuntimeWarning: divide by zero encountered in true_divide
  adjusted = (relative_freq - self.fpr_[cls]) / float(self.tpr_[cls] - self.fpr_[cls])
/usr/local/lib/python3.5/dist-packages/quantification/cc/base.py:307: RuntimeWarning: invalid value encountered in true_divide
  adjusted = (relative_freq - self.fpr_[cls]) / float(self.tpr_[cls] - self.fpr_[cls])
 81%|████████▏ | 35/43 [03:47<00:51,  6.49s/it]/home/castano/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
 86%|████████▌ | 37/43 [04:34<00:44,  7.43s/it]/usr/local/lib/python3.5/dist-packages/quantification/cc/base.py:307: RuntimeWarning: divide by zero encountered in true_divide
  adjusted = (relative_freq - self.fpr_[cls]) / float(self.tpr_[cls] - self.fpr_[

## AC - overfit

In [7]:
from quantification.cc import AC

ac = AC(estimator_class=RandomForestClassifier(n_estimators=50, max_depth=5))
for dname, dfile in tqdm(zip(dataset_names, dataset_files), total=n_datasets):
    
    X_train, X_test, y_train, y_test = load_data(dfile)
    
    ac.fit(X_train, y_train, cv=1)
    for X_test_, y_test_, prev_true, in create_bags_with_multiple_prevalence(X_test, y_test, 100):
        prev_true = prev_true[1]
        prev_pred = ac.predict(X_test_)[1]

        kld = binary_kl_divergence(prev_true, prev_pred)
        mae = absolute_error(prev_true, prev_pred)
 
        errors_df = errors_df.append(pd.DataFrame([[dname, 'ACof', prev_true, prev_pred, kld, mae, ac.tpr_[1], ac.fpr_[1]]], columns=columns))

 42%|████▏     | 18/43 [00:29<00:41,  1.66s/it]/usr/local/lib/python3.5/dist-packages/quantification/cc/base.py:307: RuntimeWarning: divide by zero encountered in true_divide
  adjusted = (relative_freq - self.fpr_[cls]) / float(self.tpr_[cls] - self.fpr_[cls])
/usr/local/lib/python3.5/dist-packages/quantification/cc/base.py:307: RuntimeWarning: invalid value encountered in true_divide
  adjusted = (relative_freq - self.fpr_[cls]) / float(self.tpr_[cls] - self.fpr_[cls])
 81%|████████▏ | 35/43 [01:10<00:16,  2.01s/it]/home/castano/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
 86%|████████▌ | 37/43 [01:27<00:14,  2.36s/it]/usr/local/lib/python3.5/dist-packages/quantification/cc/base.py:307: RuntimeWarning: divide by zero encountered in true_divide
  adjusted = (relative_freq - self.fpr_[cls]) / float(self.tpr_[cls] - self.fpr_[

In [8]:
errors_df.reset_index(inplace=True, drop=True)

In [9]:
errors_df.groupby(['method'])[['kld', 'mae']].agg(['mean', 'median'])

kld             mae        
             mean  median    mean  median
method                                   
AC50folds 1.67501 0.00624 0.10507 0.04202
ACof      1.20477 0.01040 0.12417 0.05469

In [10]:
from bokeh.plotting import figure, output_notebook, show
import seaborn as sns
import itertools
output_notebook()

palette = sns.palettes.color_palette('colorblind', n_datasets)
colors = itertools.cycle(palette.as_hex())
p = figure(plot_width=800, plot_height=1400)

for dname, color in zip(dataset_names, colors):
    df = errors_df[np.logical_and(errors_df.dataset==dname, errors_df.method=="AC50folds")]
    p.scatter(df['truth'], df['mae'], color=color, legend=dname)

p.legend.location = "top_left"
p.legend.click_policy="hide"


Loading BokehJS ...

In [11]:
show(p)

In [12]:
from bokeh.plotting import figure, output_notebook, show
import seaborn as sns
import itertools
output_notebook()

palette = sns.palettes.color_palette('colorblind', n_datasets)
colors = itertools.cycle(palette.as_hex())
p = figure(plot_width=800, plot_height=1400)

for dname, color in zip(dataset_names, colors):
    df = errors_df[np.logical_and(errors_df.dataset==dname, errors_df.method=="ACof")]
    p.scatter(df['truth'], df['mae'], color=color, legend=dname)

p.legend.location = "top_left"
p.legend.click_policy="hide"



Loading BokehJS ...

In [13]:
show(p)

In [14]:
errors_df[np.logical_and(errors_df.method=="ACof", errors_df.dataset=="semeion.8")]

,dataset,method,truth,predictions,kld,mae,tpr,fpr
4900,semeion.8,ACof,0.15193,0.04759,0.11247,0.10434,0.82927,0.00000
4901,semeion.8,ACof,0.77590,0.21089,1.05121,0.56501,0.82927,0.00000
4902,semeion.8,ACof,0.60356,0.17845,0.64429,0.42511,0.82927,0.00000
4903,semeion.8,ACof,0.39198,0.10274,0.41585,0.28923,0.82927,0.00000
4904,semeion.8,ACof,0.34327,0.10058,0.30998,0.24269,0.82927,0.00000
4905,semeion.8,ACof,0.48888,0.14925,0.46113,0.33963,0.82927,0.00000
4906,semeion.8,ACof,0.48418,0.16980,0.37779,0.31438,0.82927,0.00000
4907,semeion.8,ACof,0.16354,0.05083,0.12316,0.11271,0.82927,0.00000
4908,semeion.8,ACof,0.24627,0.05948,0.26404,0.18679,0.82927,0.00000
4909,semeion.8,ACof,0.35989,0.11140,0.30597,0.24849,0.82927,0.00000


In [16]:
errors_df.groupby(["method", "dataset"])[["tpr", "fpr", "mae"]].mean()

tpr     fpr     mae
method    dataset                                   
AC50folds acute.a            1.00000 0.00000 0.00365
          acute.b            0.94118 0.00000 0.03056
          balance.1          0.87000 0.09000 0.01396
          balance.2          0.00000 0.00000 0.57980
          balance.3          0.83000 0.10000 0.02071
          breast-cancer      0.97000 0.05000 0.00739
          cmc.1              0.48700 0.19300 0.08218
          cmc.2              0.14000 0.03571 0.25045
          cmc.3              0.13500 0.08200 0.18855
          coil               0.00000 0.00000 0.50567
          ctg.1              0.98800 0.29667 0.02516
          ctg.2              0.59000 0.03227 0.02282
          ctg.3              0.72000 0.00348 0.04032
          default_credit     0.29029 0.04051 0.05385
          diabetes           0.55000 0.10667 0.04374
          german             0.89833 0.56333 0.05275
          haberman           0.29032 0.16129 0.36632
          ionosphere         0.93023 0.03488 0.02712
          iris.1             1.00000 0.00000 0.00274
          iris.2             0.86667 0.00000 0.05748
          iris.3             0.92308 0.00000 0.02977
          lettersG           0.00800 0.00000 0.27598
          lettersH           0.17800 0.00000 0.05007
          mammographic       0.76667 0.13667 0.06259
          normtrans          0.24000 0.06500 0.09607
          normwine.1         0.88889 0.00000 0.05506
          normwine.2         0.90000 0.02500 0.02024
          normwine.3         1.00000 0.00000 0.03982
          pageblocks.5       0.47368 0.00187 0.01133
          phoneme            0.68756 0.10462 0.02197
...                              ...     ...     ...
ACof      default_credit     0.31192 0.03204 0.06292
          diabetes           0.88462 0.05263 0.12966
          german             0.97475 0.26471 0.18984
          haberman           0.67742 0.00000 0.14740
          ionosphere         0.97674 0.00000 0.04333
          iris.1             1.00000 0.00000 0.00318
          iris.2             1.00000 0.00000 0.02163
          iris.3             1.00000 0.00000 0.08712
          lettersG           0.00000 0.00000 0.49345
          lettersH           0.15984 0.00000 0.02302
          mammographic       0.83065 0.08800 0.03085
          normtrans          0.54000 0.00575 0.27196
          normwine.1         1.00000 0.00000 0.02203
          normwine.2         1.00000 0.00000 0.00998
          normwine.3         1.00000 0.00000 0.03543
          pageblocks.5       0.73684 0.00000 0.23368
          phoneme            0.77686 0.08091 0.03117
          semeion.8          0.82927 0.00000 0.34033
          sonar              1.00000 0.00000 0.16159
          spambase           0.85768 0.02719 0.00732
          spectf             1.00000 0.00000 0.42235
          tictactoe          0.91176 0.00541 0.08965
          transfusion        0.30435 0.00000 0.22872
          wdbc               0.98571 0.00000 0.03915
          wine-quality-red   0.86235 0.11207 0.06272
          wine-quality-white 0.91511 0.29821 0.07694
          wine.1             1.00000 0.00000 0.02298
          wine.2             1.00000 0.00000 0.00953
          wine.3             1.00000 0.00000 0.03141
          yeast              0.57143 0.02885 0.14661

[86 rows x 3 columns]